In [1]:
import spacy
import os
import sys
data_path = "data/wablieft/text/plain/"
sys.path.insert(0, "./skweak_git")
import skweak
nlp = spacy.load("nl_core_news_md")

In [2]:
files = [file for file in os.listdir(data_path)]
texts = []
for file in files:
    with open(data_path + file, "r", encoding="cp850") as file_text:
        texts.append(file_text.read())

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/wablieft/text/plain/'

In [3]:
docs = list(nlp.pipe(texts))

NameError: name 'texts' is not defined

In [19]:
doc = nlp("Ik woon op Straatnaam 34, in Amsterdam. Mooie straat ik woon er al erg lang. Ik werk voor Microsoft. Ik heet William Bos. Verder ben ik 34 jaar oud.")

In [24]:
def name_detector_fn(doc):
    names = ["William", "Bos"]
    for token in doc: 
        if token.text in names:
            yield token.i, token.i+1, "NAME"

name_detector = skweak.heuristics.FunctionAnnotator("name_detector", name_detector_fn)

def address_detector_fn(doc):
    for token in doc: 
        if token.text[0].isupper() and doc[token.i+1].is_digit:
            yield token.i, token.i+2, "LOCATION"

address_detector = skweak.heuristics.FunctionAnnotator("address_detector", address_detector_fn)

def company_detector_fn(doc):
    companies = ["Microsoft", "Apple", "Gemeente Amsterdam", "Universiteit van Amsterdam", "UvA"]
    for token in doc:
        if token.text in companies:
            yield token.i, token.i+1, "ORG"

company_detector = skweak.heuristics.FunctionAnnotator("company_detector", company_detector_fn)

In [28]:
doc = company_detector(name_detector(address_detector(doc)))
hmm = skweak.aggregation.HMM("hmm", ["NAME", "LOCATION", "ORG"])
hmm.fit_and_aggregate([doc])
skweak.utils.display_entities(doc, "hmm")

Starting iteration 1
Finished E-step with 1 documents
Starting iteration 2
Finished E-step with 1 documents
Starting iteration 3
Finished E-step with 1 documents
Starting iteration 4
Finished E-step with 1 documents


         1         -27.1535             +nan
         2         -26.8554          +0.2981
         3         -26.3767          +0.4787
         4         -26.1340          +0.2427


In [2]:
import spacy, re
from skweak import heuristics, gazetteers, aggregation, utils

# LF 1: heuristic to detect occurrences of MONEY entities
def money_detector(doc):
    for tok in doc[1:]:
        if tok.text[0].isdigit() and tok.nbor(-1).is_currency:
            yield tok.i-1, tok.i+1, "MONEY"
lf1 = heuristics.FunctionAnnotator("money", money_detector)

# LF 2: detection of years with a regex
lf2= heuristics.TokenConstraintAnnotator ("years", lambda tok: re.match("(19|20)\d{2}$", tok.text), "DATE")

# LF 3: a gazetteer with a few names
NAMES = [("Barack", "Obama"), ("Donald", "Trump"), ("Joe", "Biden")]
trie = gazetteers.Trie(NAMES)
lf3 = gazetteers.GazetteerAnnotator("presidents", {"PERSON":trie})

# We create a corpus (here with a single text)
nlp = spacy.load("en_core_web_sm")
doc = nlp("Donald Trump paid $750 in federal income taxes in 2016")

# apply the labelling functions
doc = lf3(lf2(lf1(doc)))

# and aggregate them
hmm = aggregation.HMM("hmm", ["PERSON", "DATE", "MONEY"])
hmm.fit_and_aggregate([doc])

# we can then visualise the final result (in Jupyter)
utils.display_entities(doc, "hmm")

Starting iteration 1
Finished E-step with 1 documents
Starting iteration 2
Finished E-step with 1 documents


         1         -18.9513             +nan
         2         -19.0673          -0.1160
